In [1]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [2]:
class Module(object):
    """
    Basically, a module is something (black box)
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`:

        output = module.forward(input)

    The module should be able to perform a backward pass: to differentiate the `forward` function.
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule.

        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True

    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.

        This includes
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput


    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.
        """

        # The easiest case:
        # self.output = input
        # return self.output
        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input.
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.

        The shape of `gradInput` is always the same as the shape of `input`.
        """
        # The easiest case:
        # self.gradInput = gradOutput
        # return self.gradInput
        pass

    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass

    def zeroGradParameters(self):
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass

    def getParameters(self):
        """
        Returns a list with its parameters.
        If the module does not have parameters - empty list.
        """
        return []

    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True

    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module to have readable description.
        """
        return "Module"


# Sequential container

Sequential — это контейнер, который позволяет объединять несколько слоев в последовательную цепочку.

То есть вместо того, чтобы вручную вызывать forward() для каждого слоя, можно использовать Sequential, и он сам вызовет forward() для всех слоев по порядку.

**Процесс прямого прохода (FORWARD PASS):**

y_0 = module[0].forward(input)      
y_1 = module[1].forward(y_0)     
...     
output = module[n-1].forward(y_{n-2})             
Нужно просто написать цикл, который передаёт выход одного слоя на вход следующему.       

backward(self, input, gradOutput)    
**Процесс обратного прохода (BACKWARD PASS):**      

g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)      
g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})      
...      
g_1 = module[1].backward(y_0, g_2)      
gradInput = module[0].backward(input, g_1)        

Каждому слою (module[i]) нужно передавать именно тот вход, который он видел на этапе forward().           
То есть не input всего Sequential, а выход предыдущего слоя!         
 


zeroGradParameters(self)
Обнуляет градиенты параметров во всех слоях.

getParameters(self)
Собирает все параметры из всех слоёв в один список.

getGradParameters(self)
Собирает все градиенты параметров из всех слоёв в один список.

__repr__(self)
Формирует текстовое представление Sequential, чтобы удобно выводить список слоёв.

**Define** a forward and backward pass procedures.

In [3]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially. 
         
         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`. 
    """
    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
   
    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:
        
            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})   
            
            
        Just write a little loop. 
        """

        # Your code goes here. ################################################
        for module in self.modules:
            input = module.forward(input)
        self.output = input
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
             
             
        !!!
                
        To ech module you need to provide the input, module saw while forward pass, 
        it is used while computing gradients. 
        Make sure that the input for `i-th` layer the output of `module[i]` (just the same input as in forward pass) 
        and NOT `input` to this Sequential module. 
        
        !!!
        
        """
        # Your code goes here. ################################################
        for i in range(len(self.modules) - 1, 0, -1):
            gradOutput = self.modules[i].backward(self.modules[i - 1].output, gradOutput)
        gradOutput = self.modules[0].backward(input, gradOutput)
        self.gradInput = gradOutput
        return self.gradInput
      

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        return self.modules.__getitem__(x)
    
    def train(self):
        """
        Propagates training parameter through all modules
        """
        self.training = True
        for module in self.modules:
            module.train()
    
    def evaluate(self):
        """
        Propagates training parameter through all modules
        """
        self.training = False
        for module in self.modules:
            module.evaluate()



# Layers

## 1 (0.2). Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

прямой и обратный проход (градиенты)

В глубоком обучении слой линейного преобразования, также известный как полносвязный слой (fully connected layer, dense layer, linear layer), является фундаментальным компонентом многих типов нейронных сетей, включая полносвязные нейронные сети, рекуррентные или сверточные.    
    
В полносвязном слое каждый нейрон текущего слоя связан со всеми нейронами предыдущего слоя. Это означает, что каждый нейрон в полносвязном слое     получает входные сигналы от всех нейронов предыдущего слоя, и каждая такая связь имеет свой весовой коэффициент.     
источник: https://habr.com/ru/articles/885466/

updateOutput: Выполняется матричное умножение входа input на транспонированную матрицу весов W и прибавляется смещение b.     
updateGradInput: Градиент по входу вычисляется через умножение градиента выхода gradOutput на веса W.       
accGradParameters: Градиенты по весам W накапливаются как произведение транспонированного gradOutput на input, а градиенты по смещению b — как сумма gradOutput по батчам.

проверка пройдена

In [4]:
class Linear(Module):
    """
    A module which applies a linear transformation
    A common name is fully-connected layer, InnerProductLayer in caffe.

    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()

        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):
        # Your code goes here. ################################################
        # Вычисляем линейное преобразование: output = input * W^T + b
         self.output = np.dot(input, self.W.T) + self.b
         return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        # Градиент по входу: gradInput = gradOutput * W
        self.gradInput = np.dot(gradOutput, self.W)
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        # Вычисляем градиенты по параметрам:
        # gradW = gradOutput^T * input
        # gradb = сумма по батчу значений gradOutput
        self.gradW += np.dot(gradOutput.T, input)
        self.gradb += np.sum(gradOutput, axis=0)

    def zeroGradParameters(self):               # обнуление градиентов на каждой итерации обучения.    
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

## 2. (0.2) SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

что читала: https://translated.turbopages.org/proxy_u/en-ru.ru.53dc90e0-67d96098-8897f6a4-74722d776562/https/www.geeksforgeeks.org/how-to-implement-softmax-and-cross-entropy-in-python-and-pytorch/

проверка пройдена

In [5]:
    class SoftMax(Module):
        def __init__(self):
            super(SoftMax, self).__init__()
    
        def updateOutput(self, input):
            self.output = np.subtract(input, input.max(axis=1, keepdims=True))
    
            np.exp(self.output, out=self.output)
            np.divide(self.output, self.output.sum(axis=1, keepdims=True), out=self.output)
            return self.output
    
        def updateGradInput(self, input, gradOutput):
            self.gradInput = gradOutput * self.output
    
            np.subtract(self.gradInput, self.output * self.gradInput.sum(axis=1, keepdims=True), out=self.gradInput)
            return self.gradInput
        def __repr__(self):
            return "SoftMax"

## 3. (0.2) LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

проверка пройдена

In [6]:
    class LogSoftMax(Module):
        def __init__(self):
            super(LogSoftMax, self).__init__()
    
        def updateOutput(self, input):
            # start with normalization for numerical stability
            self.output = np.subtract(input, input.max(axis=1, keepdims=True))
    
            np.subtract(self.output, np.log(np.exp(self.output).sum(axis=1, keepdims=True)), out=self.output)
            return self.output
    
        def updateGradInput(self, input, gradOutput):
            self.gradInput = gradOutput - np.exp(self.output) * np.sum(gradOutput, axis=1, keepdims=True)       
            return self.gradInput
    
        def __repr__(self):
            return "LogSoftMax"

## 4. (0.3) Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance:
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance.

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

что читала: https://habr.com/ru/companies/mvideo/articles/782360/ 

В основе Batch Normalization лежит решение проблемы «внутреннего ковариационного сдвига» (Internal Covariate Shift). Этот термин описывает явление, при котором распределение входных данных каждого слоя нейронной сети меняется в процессе обучения, из-за чего сети становится сложнее обучать. Это происходит из-за того, что параметры предыдущих слоев изменяются во время обучения, влияя на данные текущего слоя.

Batch Normalization решает эту проблему, нормализуя выход каждого слоя. Нормализация заключается в преобразовании входных данных каждого слоя таким образом, чтобы среднее значение было приближено к нулю, а стандартное отклонение — к единице. Это делает сеть менее чувствительной к масштабу входных данных и улучшает общую стабильность процесса обучения.


проверка пройдена

In [7]:

class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None 
        self.moving_variance = None
      
    def updateMeanAndVariance(self, batch_mean, batch_variance):
        self.moving_mean = batch_mean if self.moving_mean is None else self.moving_mean
        self.moving_mean = self.alpha * self.moving_mean + (1 - self.alpha) * batch_mean

        self.moving_variance = batch_variance if self.moving_variance is None else self.moving_variance
        self.moving_variance = self.alpha * self.moving_variance + (1 - self.alpha) * batch_variance
        
    def updateOutput(self, input):
        batch_mean = np.mean(input, axis=0) if self.training else self.moving_mean
        batch_variance = np.var(input, axis=0) if self.training else self.moving_variance

        self.output = (input - batch_mean) / np.sqrt(batch_variance + self.EPS)
        if self.training:
          self.updateMeanAndVariance(batch_mean, batch_variance) 

        return self.output
    
    def updateGradInput(self, input, gradOutput):
        batch_mean = np.mean(input, axis=0) if self.training else self.moving_mean
        batch_variance = np.var(input, axis=0) if self.training else self.moving_variance

        n = input.shape[0]

        part_var = -np.sum(gradOutput * (input - batch_mean), axis=0) / (2 * (batch_variance + self.EPS) ** (3/2))
        part_mean = - np.sum(gradOutput / np.sqrt(batch_variance + self.EPS), axis=0) - 2 * part_var * np.sum(input - batch_mean, axis=0) / n
        self.gradInput = gradOutput / np.sqrt(batch_variance + self.EPS) + 2 / n * part_var * (input - batch_mean) + part_mean / n

        return self.gradInput
    
    def __repr__(self):
        return "BatchNormalization"

масштабирование по каналам уже реализовано

In [8]:
    class ChannelwiseScaling(Module):
        """
           Implements linear transform of input y = \gamma * x + \beta
           where \gamma, \beta - learnable vectors of length x.shape[-1]
        """
        def __init__(self, n_out):
            super(ChannelwiseScaling, self).__init__()
    
            stdv = 1./np.sqrt(n_out)
            self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
            self.beta = np.random.uniform(-stdv, stdv, size=n_out)
    
            self.gradGamma = np.zeros_like(self.gamma)
            self.gradBeta = np.zeros_like(self.beta)
    
        def updateOutput(self, input):
            self.output = input * self.gamma + self.beta
            return self.output
    
        def updateGradInput(self, input, gradOutput):
            self.gradInput = gradOutput * self.gamma
            return self.gradInput
    
        def accGradParameters(self, input, gradOutput):
            self.gradBeta = np.sum(gradOutput, axis=0)
            self.gradGamma = np.sum(gradOutput*input, axis=0)
    
        def zeroGradParameters(self):
            self.gradGamma.fill(0)
            self.gradBeta.fill(0)
    
        def getParameters(self):
            return [self.gamma, self.beta]
    
        def getGradParameters(self):
            return [self.gradGamma, self.gradBeta]
    
        def __repr__(self):
            return "ChannelwiseScaling"

<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Ксения\AppData\Local\Temp\ipykernel_35392\3971934634.py:2: SyntaxWarning: invalid escape sequence '\g'
  """


Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.



Практические заметки. Если пакетная нормализация размещена после слоя линейного преобразования (включая плотный слой, свертки, масштабирование по каналам), который реализует функцию типа y = weight * x + bias, то добавление смещения становится бесполезным и может быть опущено, поскольку его эффект будет отброшен при вычитании пакетного среднего. Если пакетная нормализация (за которой следует масштабирование по каналам) помещается перед слоем, который увеличивает масштаб (включая ReLU, LeakyReLU), за которым следует любой слой линейного преобразования, то параметр gamma в масштабировании по каналам может быть заблокирован, поскольку он может быть поглощен слоем линейного преобразования.

## 5. (0.3) Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

проверка пройдена

In [9]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        
        self.p = p
        self.mask = None
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        if self.training:
            self.mask = np.random.binomial(1, 1 - self.p, size=input.shape) / (1 - self.p)
        else:
            self.mask = np.ones(input.shape)
        self.output = input * self.mask
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = gradOutput * self.mask
        return self.gradInput
        
    def __repr__(self):
        return "Dropout"


## 6. (2.0) Conv2d       
Implement [**Conv2d**](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html). Use only this list of parameters: (in_channels, out_channels, kernel_size, stride, padding, bias, padding_mode) and fix dilation=1 and groups=1.

что читала: https://proproprogs.ru/nn_pytorch/pytorch-klassy-conv2d-i-maxpool2d?ysclid=m8elvqk8zt642465353

np.prod — это функция из библиотеки NumPy, которая используется для вычисления произведения всех элементов массива или элементов вдоль указанной оси.


проверку не прошла, сложный пункт :(((


In [10]:
import scipy as sp
import scipy.signal
import skimage

class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(Conv2d, self).__init__()
        assert kernel_size % 2 == 1, kernel_size

        stdv = 1./np.sqrt(in_channels)
        self.W = np.random.uniform(-stdv, stdv, size = (out_channels, in_channels, kernel_size, kernel_size))
        self.b = np.random.uniform(-stdv, stdv, size=(out_channels,))
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):
        pad_size = self.kernel_size // 2

        input_padded = skimage.util.pad(input, ((0, 0), (0, 0), (pad_size, pad_size), (pad_size, pad_size)), mode='constant')

        batch_size, in_channels, h, w = input.shape
        out_h = h
        out_w = w
        self.output = np.zeros((batch_size, self.out_channels, out_h, out_w))

        for b in range(batch_size):
            for out_c in range(self.out_channels):
                for in_c in range(self.in_channels):
                    self.output[b, out_c, :, :] += scipy.signal.correlate(input_padded[b, in_c], self.W[out_c, in_c], mode='valid')

                self.output[b, out_c, :, :] += self.b[out_c]
        # 1. zero-pad the input array
        # 2. compute convolution using scipy.signal.correlate(... , mode='valid')
        # 3. add bias value

        # self.output = ...


        return self.output

    def updateGradInput(self, input, gradOutput):
        pad_size = self.kernel_size // 2

        gradOutput_padded = skimage.util.pad(gradOutput, ((0, 0), (0, 0), (pad_size, pad_size), (pad_size, pad_size)), mode='constant')

        self.gradInput = np.zeros_like(input)

        for b in range(input.shape[0]):
            for in_c in range(self.in_channels):
                for out_c in range(self.out_channels):
                  rotated_kernel = np.rot90(self.W[out_c, in_c], 2)

                  self.gradInput[b, in_c] += scipy.signal.correlate(
                      gradOutput_padded[b, out_c], rotated_kernel, mode='valid')
        # 1. zero-pad the gradOutput
        # 2. compute 'self.gradInput' value using scipy.signal.correlate(... , mode='valid')

        # self.gradInput = ...

        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        pad_size = self.kernel_size // 2

        input_padded = skimage.util.pad(input, ((0, 0), (0, 0), (pad_size, pad_size), (pad_size, pad_size)), mode='constant')

        gradOutput_padded = skimage.util.pad(gradOutput, ((0, 0), (0, 0), (pad_size, pad_size), (pad_size, pad_size)), mode='constant')

        self.gradW.fill(0)
        self.gradb.fill(0)

        for b in range(input.shape[0]):
            for out_c in range(self.out_channels):
                for in_c in range(self.in_channels):
                    self.gradW[out_c, in_c] += scipy.signal.correlate(input_padded[b, in_c], gradOutput_padded[b, out_c], mode='valid')
                self.gradb[out_c] += np.sum(gradOutput[b, out_c], axis=(0, 1))

        # 1. zero-pad the input
        # 2. compute 'self.gradW' using scipy.signal.correlate(... , mode='valid')
        # 3. compute 'self.gradb' - formulas like in Linear of ChannelwiseScaling layers

        # self.gradW = ...
        # self.gradb = ...
        pass

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        q = 'Conv2d %d -> %d' %(s[1],s[0])
        return q

## 7. (0.5) Implement 
[**MaxPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) and [**AvgPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html). Use only parameters like kernel_size, stride, padding (negative infinity for maxpool and zero for avgpool) and other parameters fixed as in framework.

что смотрела: https://www.geeksforgeeks.org/apply-a-2d-max-pooling-in-pytorch/?ysclid=m8eu2wkps8639082364

тут проверку прошла 


In [11]:
    class MaxPool2d(Module):
        def __init__(self, kernel_size, stride, padding):
            super(MaxPool2d, self).__init__()
    
            self.kernel_size = kernel_size
            self.stride = stride
            self.padding = padding
            self.indices = None  # Сохраняем индексы максимальных значений
    
        def updateOutput(self, input):
            batch_size, channels, height, width = input.shape
            out_height = (height - self.kernel_size) // self.stride + 1
            out_width = (width - self.kernel_size) // self.stride + 1
    
            self.output = np.zeros((batch_size, channels, out_height, out_width))
            self.indices = np.zeros_like(input, dtype=bool)
    
            for i in range(out_height):
                for j in range(out_width):
                    region = input[:, :, i * self.stride:i * self.stride + self.kernel_size,
                                   j * self.stride:j * self.stride + self.kernel_size]
                    self.output[:, :, i, j] = np.max(region, axis=(2, 3))
                    
                    # Сохраняем маску позиций максимальных значений
                    max_mask = (region == self.output[:, :, i, j][:, :, None, None])
                    self.indices[:, :, i * self.stride:i * self.stride + self.kernel_size,
                                 j * self.stride:j * self.stride + self.kernel_size] |= max_mask
    
            return self.output
    
        def updateGradInput(self, input, gradOutput):
            self.gradInput = np.zeros_like(input)
            for i in range(gradOutput.shape[2]):
                for j in range(gradOutput.shape[3]):
                    self.gradInput[:, :, i * self.stride:i * self.stride + self.kernel_size,
                                   j * self.stride:j * self.stride + self.kernel_size] += (
                        self.indices[:, :, i * self.stride:i * self.stride + self.kernel_size,
                                     j * self.stride:j * self.stride + self.kernel_size] * 
                        gradOutput[:, :, i, j][:, :, None, None]
                    )
            return self.gradInput
        def __repr__(self):
            return "MaxPool2d"
    
    class AvgPool2d(Module):
        def __init__(self, kernel_size, stride, padding=0):
            super(AvgPool2d, self).__init__()

            self.kernel_size = kernel_size
            self.stride = stride
            self.padding = padding

        def pad_input(self, input):
            """Добавляет паддинг, если необходимо."""
            if self.padding > 0:
                return np.pad(input, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode='constant', constant_values=0)
            return input

        def updateOutput(self, input):
            input_padded = self.pad_input(input)
            batch_size, channels, height, width = input_padded.shape

            # Вычисляем правильные размеры выхода
            out_height = (height - self.kernel_size) // self.stride + 1
            out_width = (width - self.kernel_size) // self.stride + 1

            self.output = np.zeros((batch_size, channels, out_height, out_width))

            for i in range(out_height):
                for j in range(out_width):
                    region = input_padded[:, :, i * self.stride:i * self.stride + self.kernel_size,
                                          j * self.stride:j * self.stride + self.kernel_size]
                    self.output[:, :, i, j] = np.mean(region, axis=(2, 3))

            return self.output

        def updateGradInput(self, input, gradOutput):
            input_padded = self.pad_input(input)
            self.gradInput = np.zeros_like(input_padded)
            factor = 1 / (self.kernel_size * self.kernel_size)

            for i in range(gradOutput.shape[2]):
                for j in range(gradOutput.shape[3]):
                    self.gradInput[:, :, i * self.stride:i * self.stride + self.kernel_size,
                                   j * self.stride:j * self.stride + self.kernel_size] += (
                        gradOutput[:, :, i, j][:, :, None, None] * factor
                    )

            # Убираем паддинг из градиента
            if self.padding > 0:
                self.gradInput = self.gradInput[:, :, self.padding:-self.padding, self.padding:-self.padding]

            return self.gradInput
        
        def __repr__(self):
            return "AvgPool2d"

## 8. (0.3) Implement **GlobalMaxPool2d** and **GlobalAvgPool2d**. They do not have testing and parameters are up to you but they must aggregate information within channels. Write test functions for these layers on your own.

In [12]:
    class GlobalMaxPool2d(Module):
        def __init__(self):
            super(GlobalMaxPool2d, self).__init__()
    
        def updateOutput(self, input):
            self.output = np.max(input, axis=(2, 3), keepdims=True)
            return self.output
    
        def updateGradInput(self, input, gradOutput):
            self.gradInput = np.zeros_like(input)
            mask = (input == self.output)
            self.gradInput[mask] = gradOutput
            return self.gradInput
    
        def __repr__(self):
            return "GlobalMaxPool2d"
    
    
    class GlobalAvgPool2d(Module):
        def __init__(self):
            super(GlobalAvgPool2d, self).__init__()
    
        def updateOutput(self, input):
            self.output = np.mean(input, axis=(2, 3), keepdims=True)
            return self.output
    
        def updateGradInput(self, input, gradOutput):
            self.gradInput = np.ones_like(input) * (gradOutput / (input.shape[2] * input.shape[3]))
            return self.gradInput
    
        def __repr__(self):
            return "GlobalAvgPool2d"

# 9. (0.2) Implement [**Flatten**](https://pytorch.org/docs/stable/generated/torch.flatten.html)

ок

In [13]:
class Flatten(Module):
    def __init__(self, start_dim=1, end_dim=-1):
        super(Flatten, self).__init__()
        self.start_dim = start_dim
        self.end_dim = end_dim

    def updateOutput(self, input):
        # Рассчитываем конечный end_dim для отрицательных индексов
        end_dim = self.end_dim if self.end_dim >= 0 else len(input.shape) + self.end_dim
        # Изменяем форму по указанным измерениям
        new_shape = input.shape[:self.start_dim] + (-1,) + input.shape[end_dim + 1:]
        self.output = input.reshape(new_shape)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput.reshape(input.shape)
        return self.gradInput

    
    
    def __repr__(self):
        return "Flatten"


# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**):

In [14]:
    class ReLU(Module):
        def __init__(self):
             super(ReLU, self).__init__()
    
        def updateOutput(self, input):
            self.output = np.maximum(input, 0)
            return self.output
    
        def updateGradInput(self, input, gradOutput):
            self.gradInput = np.multiply(gradOutput , input > 0)
            return self.gradInput
    
        def __repr__(self):
            return "ReLU"

## 10. (0.1) Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

In [15]:
 class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()
            
        self.slope = slope
        
    def updateOutput(self, input):
        
        self.output = np.maximum(input,self.slope*input)
    
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        #self.gradInput = np.ones_like(input)
        #self.gradInput[input < 0] = self.slope
        #self.gradInput = self.gradInput*gradOutput
        self.gradInput = np.multiply(gradOutput, input > 0) + np.multiply(gradOutput * self.slope, input < 0)
        return self.gradInput
    
    def __repr__(self):
        return "LeakyReLU"

## 11. (0.1) ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [16]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__()
        
        self.alpha = alpha
        
    def updateOutput(self, input):
        #z if z >= 0 else alpha*(e^z -1)
        pos_num = np.maximum(input, 0)
        neg_num = input - pos_num
        neg_num = self.alpha*(np.exp(neg_num) -1)
        self.output = pos_num + neg_num
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        pos_num = np.maximum(input, 0)
        neg_num = input - pos_num
        neg_num = self.alpha*(np.exp(neg_num) -1)+self.alpha
    
        neg_num[pos_num>0] = 1
        self.gradInput = neg_num*gradOutput
       
        return self.gradInput
    
    def __repr__(self):
        return "ELU"

## 12. (0.1) SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

тут всё ок

In [17]:
    class SoftPlus(Module):
        def __init__(self):
            super(SoftPlus, self).__init__()
        
        def updateOutput(self, input):
            # Your code goes here. ################################################
            self.output = np.log(np.exp(input) + 1)
            
            return  self.output
        
        def updateGradInput(self, input, gradOutput):
            # Your code goes here. ################################################
            div_exp_input = np.exp(input) / (np.exp(input) + 1)
            self.gradInput = np.multiply(gradOutput, div_exp_input)
            
            return self.gradInput
        
        def __repr__(self):
            return "SoftPlus"

#### 13. (0.2) Gelu
Implement [**Gelu**](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html) activations.

In [18]:

class Gelu(Module):
    def __init__(self):
        super(Gelu, self).__init__()

    def updateOutput(self, input):
        # Точное вычисление GELU, как в PyTorch
        self.output = 0.5 * input * (1 + np.tanh(np.sqrt(2 / np.pi) * 
                          (input + 0.044715 * input**3)))
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Производная GELU
        tanh_term = np.tanh(np.sqrt(2 / np.pi) * (input + 0.044715 * input**3))
        sech2_term = 1 - tanh_term**2  # производная tanh(x)
        grad_gelu = 0.5 * (1 + tanh_term) + input * 0.5 * sech2_term * (
            np.sqrt(2 / np.pi) * (1 + 3 * 0.044715 * input**2)
        )
        # Умножение на gradOutput для обратного прохода
        self.gradInput = gradOutput * grad_gelu
        return self.gradInput

    def __repr__(self):
        return "Gelu"


# Criterions

Criterions are used to score the models answers.

In [19]:
class Criterion(object):
    def init (self):
        self.output = None
        self.gradInput = None

    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in updateOutput.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in updateGradInput.
        """
        return self.updateGradInput(input, target)

    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput

    def repr(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [20]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, input, target):
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 14. (0.2) Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula,
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [21]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def init(self):
        a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).init()

    def updateOutput(self, input, target):

        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)

        self.output = -np.sum(target * np.log(input_clamp)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):

        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)

        self.gradInput = -target / input_clamp / input.shape[0]
        return self.gradInput

    def repr(self):
        return "ClassNLLCriterionUnstable"

## 15. (0.3) Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [22]:
class ClassNLLCriterion(Criterion):
    def init(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).init()

    def updateOutput(self, input, target):
        self.output = -np.sum(target * input) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput = -target / input.shape[0]
        return self.gradInput

    def repr(self):
        return "ClassNLLCriterion"

#### ну вроде все проверки прошла, кроме conv2d.

1-я часть задания: реализация слоев, лосей и функций активации - 5 баллов. \\
2-я часть задания: реализация моделей на своих классах. Что должно быть:
  1. Выберите оптимизатор и реализуйте его, чтоб он работал с вами классами. - 1 балл.
  2. Модель для задачи мультирегрессии на выбраных вами данных. Использовать FCNN, dropout, batchnorm, MSE. Пробуйте различные фукнции активации. Для первой модели попробуйте большую, среднюю и маленькую модель. - 1 балл.
  3. Модель для задачи мультиклассификации на MNIST. Использовать свёртки, макспулы, флэттэны, софтмаксы - 1 балла.
  4. Автоэнкодер для выбранных вами данных. Должен быть на свёртках и полносвязных слоях, дропаутах, батчнормах и тд. - 2 балла. \\

Дополнительно в оценке каждой модели будет учитываться:
1. Наличие правильно выбранной метрики и лосс функции.
2. Отрисовка графиков лосей и метрик на трейне-валидации. Проверка качества модели на тесте.
3. Наличие шедулера для lr.
4. Наличие вормапа.
5. Наличие механизма ранней остановки и сохранение лучшей модели.
6. Свитч лося (метрики) и оптимайзера.

### 2. часть

### 1 Выберите оптимизатор и реализуйте его, чтоб он работал с вами классами. - 1 балл.

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate, KFold, GridSearchCV, StratifiedShuffleSplit


Реализация оптимизатора Adam.        
variables: список с текущими значениями параметров модели.     
gradients: список с градиентами для каждого параметра модели.      
config: конфигурация с гиперпараметрами (learning_rate, beta1, beta2, epsilon).     
state: состояние оптимизатора, включающее моменты и счетчик шагов.        

In [24]:
class MyModel:
    def __init__(self):
        self.weights = np.random.randn()  # случайный вес
        self.bias = np.random.randn()     # случайное смещение

    def parameters(self):
        return [self.weights, self.bias]

    def forward(self, x):
        return self.weights * x + self.bias

    def backward(self, x, y): #взяла линейную регрессию
        # Тут градиенты по весам и смещениям
        self.weights_grad = 2 * (self.forward(x) - y) * x  # градиент по весам
        self.bias_grad = 2 * (self.forward(x) - y)         # градиент по смещению
        return [self.weights_grad, self.bias_grad]


In [25]:
def adam_optimizer(variables, gradients, config, state):
   
    state.setdefault('m', {})  # Первый момент (моменты первого порядка)
    state.setdefault('v', {})  # Второй момент (моменты второго порядка)
    state.setdefault('t', 0)   # Шаг оптимизации
    state['t'] += 1
    
    # Проверка наличия гиперпараметров
    for k in ['learning_rate', 'beta1', 'beta2', 'epsilon']:
        assert k in config, f"Missing configuration key: {k}"

    lr_t = config['learning_rate'] * np.sqrt(1 - config['beta2']**state['t']) / (1 - config['beta1']**state['t'])
    
    var_index = 0
    for current_var, current_grad in zip(variables, gradients):
        # Инициализация моментов
        var_first_moment = state['m'].setdefault(var_index, np.zeros_like(current_grad))
        var_second_moment = state['v'].setdefault(var_index, np.zeros_like(current_grad))

        # Обновляем моменты
        np.add(config['beta1'] * var_first_moment, (1 - config['beta1']) * current_grad, out=var_first_moment)
        np.add(config['beta2'] * var_second_moment, (1 - config['beta2']) * current_grad ** 2, out=var_second_moment)

        # Обновление параметра с использованием моментов
        current_var -= lr_t * var_first_moment / (np.sqrt(var_second_moment) + config['epsilon'])

        # Переход к следующему параметру
        var_index += 1


In [26]:
# Конфигурация для оптимизатора
config = {
    'learning_rate': 0.01,
    'beta1': 0.9,
    'beta2': 0.999,
    'epsilon': 1e-8
}

# Состояние оптимизатора (инициализируется пустым)
state = {}

# Создаем модель
model = MyModel()

for epoch in range(100):  # 100 эпох обучения
    # Генерируем случайные входные данные и целевые значения
    x = np.random.randn()  # случайное значение для x
    y = 2 * x + 1          # целевое значение y = 2x + 1

    # Получаем градиенты от модели
    gradients = model.backward(x, y)
    
    # Получаем параметры модели
    variables = model.parameters()
    
    # Обновляем параметры с использованием оптимизатора Adam
    adam_optimizer(variables, gradients, config, state)
    
    # Каждые 10 эпох выводим текущие значения параметров
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Weights: {model.weights}, Bias: {model.bias}")


Epoch 0, Weights: 0.4317842415715972, Bias: -0.08760178298416667
Epoch 10, Weights: 0.4317842415715972, Bias: -0.08760178298416667
Epoch 20, Weights: 0.4317842415715972, Bias: -0.08760178298416667
Epoch 30, Weights: 0.4317842415715972, Bias: -0.08760178298416667
Epoch 40, Weights: 0.4317842415715972, Bias: -0.08760178298416667
Epoch 50, Weights: 0.4317842415715972, Bias: -0.08760178298416667
Epoch 60, Weights: 0.4317842415715972, Bias: -0.08760178298416667
Epoch 70, Weights: 0.4317842415715972, Bias: -0.08760178298416667
Epoch 80, Weights: 0.4317842415715972, Bias: -0.08760178298416667
Epoch 90, Weights: 0.4317842415715972, Bias: -0.08760178298416667


### 2. Модель для задачи мультирегрессии на выбраных вами данных. Использовать FCNN, dropout, batchnorm, MSE. Пробуйте различные фукнции активации. Для первой модели попробуйте большую, среднюю и маленькую модель. - 1 балл.


### Источник:
Набор данных был создан Ангелики Шифара (angxifara '@' gmail.com, инженер-строитель/строитель) и обработан Атанасиосом Цанасом (tsanasthanasis '@' gmail.com, Оксфордский центр промышленной и прикладной математики, Оксфордский университет, Великобритания).

### Информация о наборе данных:
Мы выполняем энергетический анализ с использованием 12 различных форм зданий, смоделированных в Ecotect. Здания различаются по площади остекления, распределению площади остекления и ориентации, а также по другим параметрам. Мы моделируем различные настройки как функции вышеупомянутых характеристик, чтобы получить 768 форм зданий. Набор данных состоит из 768 образцов и 8 признаков, направленных на прогнозирование двух вещественных реакций. Его также можно использовать в качестве задачи многоклассовой классификации, если реакцию округлить до ближайшего целого числа.

### Информация о характеристиках:
Набор данных содержит восемь атрибутов (или признаков, обозначенных X1... X8) и два ответа (или результата, обозначаемых y1 и y2). Цель состоит в том, чтобы использовать восемь признаков для прогнозирования каждого из двух ответов.

В частности:
x1 относительная компактность x2 площадь поверхности x3 площадь стены x4 площадь крыши x5 общая высота x6 ориентация x7 площадь остекления x8 распределение площади остекления y1 тепловая нагрузка y2 охлаждающая нагрузка



In [27]:
data = pd.read_csv('data.csv')

In [28]:
data

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28
...,...,...,...,...,...,...,...,...,...,...
763,0.64,784.0,343.0,220.50,3.5,5,0.4,5,17.88,21.40
764,0.62,808.5,367.5,220.50,3.5,2,0.4,5,16.54,16.88
765,0.62,808.5,367.5,220.50,3.5,3,0.4,5,16.44,17.11
766,0.62,808.5,367.5,220.50,3.5,4,0.4,5,16.48,16.61


In [29]:
null_data = data[data.isnull().any(axis=1)]
null_data

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


In [30]:
data[data.duplicated()]

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


In [31]:
X = data.drop(['Y1','Y2'], axis=1)
y = data[['Y1','Y2']]

In [32]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, shuffle = True) #shuffle - перемешиваем данные перед разделением
X_test.shape 

(192, 8)

In [39]:
X_train = np.array(X_train)  # Если X_train в pandas DataFrame, преобразуем в numpy
y_train = np.array(y_train)  # То же для y_train
X_test = np.array(X_test)
y_test = np.array(y_test)

In [42]:
print(X_train.shape)  # (100, 16)
print(y_train.shape)  # (100,)

(576, 8)
(576, 2)


In [40]:
from sklearn.preprocessing import StandardScaler


In [41]:
def build_model(size="small"):
    model = Sequential()
    input_dim = X_train.shape[1]  # 8 входных признаков
    
    if size == "small":
        model.add(Linear(input_dim, 16))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(Linear(16, 2))  # Два выхода (Y1, Y2)
    
    elif size == "medium":
        model.add(Linear(input_dim, 32))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(Dropout(0.2))
        model.add(Linear(32, 16))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(Linear(16, 2))
    
    elif size == "large":
        model.add(Linear(input_dim, 64))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(Dropout(0.3))
        model.add(Linear(64, 32))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(Dropout(0.2))
        model.add(Linear(32, 16))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(Linear(16, 2))
    
    return model

# Функция обучения

def train_model(model, X_train, y_train, epochs=100, lr=0.01):
    for epoch in range(epochs):
        # Прямой проход
        y_pred = model.forward(X_train)
        
        # Вычисление ошибки
        criterion = MSECriterion()
        loss = criterion.forward(y_pred, y_train)
        
        # Обратный проход
        model.zeroGradParameters()  # Обнуляем старые градиенты
        grad_output = criterion.backward(y_pred, y_train)
        model.backward(X_train, grad_output)
        
        # Градиентный шаг (SGD)
        for i, (param, grad) in enumerate(zip(model.getParameters(), model.getGradParameters())):
            if grad is not None:
                try:
                    # Проверяем, что параметры и градиенты имеют правильную форму
                    print(f"Layer {i}:")
                    print(f"  Param shape: {np.shape(param)}")
                    print(f"  Grad shape: {np.shape(grad)}")

                    param = np.array(param)  # Преобразуем в NumPy
                    grad = np.array(grad)    # Преобразуем в NumPy
                    
                    # Обновление весов и смещений
                    param -= lr * grad
                    
                except Exception as e:
                    print(f"Error updating param {i}: {e}")

        # Логируем каждые 10 эпох
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss}")




# Оценка модели
def evaluate_model(model, X_test, y_test):
    predictions = model.forward(X_test)
    loss = MSECriterion().forward(predictions, y_test)
    print(f"Test Loss: {loss}")

# Обучаем и тестируем модели
for size in ["small", "medium", "large"]:
    print(f"Training {size} model...")
    model = build_model(size)
    train_model(model, X_train, y_train)
    evaluate_model(model, X_test, y_test)
    print("-" * 50)

Training small model...
Layer 0:
Error updating param 0: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 16) + inhomogeneous part.
Layer 1:
  Param shape: (0,)
  Grad shape: (0,)
Layer 2:
  Param shape: (0,)
  Grad shape: (0,)
Layer 3:
Error updating param 3: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 2) + inhomogeneous part.
Epoch 0, Loss: 1350.9726830627058
Layer 0:
Error updating param 0: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 16) + inhomogeneous part.
Layer 1:
  Param shape: (0,)
  Grad shape: (0,)
Layer 2:
  Param shape: (0,)
  Grad shape: (0,)
Layer 3:
Error updating param 3: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (